In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
df = pd.read_csv("C:/Users/pkitt/Downloads/playground-series-s5e6/train.csv")

In [4]:
df.cat =df.select_dtypes(include=[object]) 

C:\Users\pkitt\AppData\Local\Temp\ipykernel_35404\1653857449.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.cat =df.select_dtypes(include=[object])


In [5]:
df.num=df.select_dtypes(include=[np.number])

C:\Users\pkitt\AppData\Local\Temp\ipykernel_35404\610118065.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.num=df.select_dtypes(include=[np.number])


In [6]:
df.cat.head()

,Soil Type,Crop Type,Fertilizer Name
0,Clayey,Sugarcane,28-28
1,Sandy,Millets,28-28
2,Sandy,Millets,17-17-17
3,Sandy,Barley,10-26-26
4,Red,Paddy,DAP


In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
le = LabelEncoder()

In [9]:
df.cat["Soil Type"] = le.fit_transform(df.cat["Soil Type"])

In [10]:
df.cat.head()

,Soil Type,Crop Type,Fertilizer Name
0,1,Sugarcane,28-28
1,4,Millets,28-28
2,4,Millets,17-17-17
3,4,Barley,10-26-26
4,3,Paddy,DAP


In [11]:
df.cat["Crop Type"] = le.fit_transform(df.cat["Crop Type"])

In [12]:
df.cat.head()

,Soil Type,Crop Type,Fertilizer Name
0,1,8,28-28
1,4,4,28-28
2,4,4,17-17-17
3,4,0,10-26-26
4,3,6,DAP


In [13]:
df.cat["Fertilizer Name"] = le.fit_transform(df.cat["Fertilizer Name"])

In [14]:
df_train = pd.concat([df.num,df.cat],axis=1)

In [15]:
import tensorflow as tf

In [16]:
from tensorflow import keras

In [17]:
X = df_train[["Temparature","Humidity","Moisture","Soil Type"]]

In [18]:
y = df_train[["Fertilizer Name"]]

In [19]:
df_sorted = df.sort_values(by='Fertilizer Name')

In [20]:
grouped_batches = [group for _, group in df_sorted.groupby('Fertilizer Name')]

In [21]:
batch_1 = X[y == 0]  # Class 0
batch_2 = X[y == 1]  # Class 1
batch_3 = X[y == 2]  # Class 2
batch_4 = X[y == 3]  # Class 3
batch_5 = X[y == 4]  # Class 4
batch_6 = X[y == 5]  # Class 5


In [22]:
y.head()

,Fertilizer Name
0,4
1,4
2,2
3,0
4,5


In [23]:
def ordered_classwise_batches(df, feature_cols, label_col, batch_size=32):
    batches = []
    class_labels = sorted(df[label_col].unique())  # Ensures order: 0, 1, 2, ...

    for label in class_labels:
        group = df[df[label_col] == label].reset_index(drop=True)
        for i in range(0, len(group), batch_size):
            batch = group.iloc[i:i + batch_size]
            X_batch = batch[feature_cols].values
            y_batch = batch[label_col].values
            batches.append((X_batch, y_batch))

    return batches

In [24]:
feature_cols = ['Nitrogen','Potassium','Moisture','Temparature','Humidity','Soil Type']
label_col = 'Fertilizer Name'

batches = ordered_classwise_batches(df_train, feature_cols, label_col, batch_size=64)

# Confirm that first few batches are in order
for i, (X, y) in enumerate(batches[:10]):
    print(f"Batch {i} - Fertilizer Label(s): {np.unique(y)}")

Batch 0 - Fertilizer Label(s): [0]
Batch 1 - Fertilizer Label(s): [0]
Batch 2 - Fertilizer Label(s): [0]
Batch 3 - Fertilizer Label(s): [0]
Batch 4 - Fertilizer Label(s): [0]
Batch 5 - Fertilizer Label(s): [0]
Batch 6 - Fertilizer Label(s): [0]
Batch 7 - Fertilizer Label(s): [0]
Batch 8 - Fertilizer Label(s): [0]
Batch 9 - Fertilizer Label(s): [0]


In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential([
    Dense(64, activation='relu', input_shape=(6,)),
    Dense(32, activation='relu'),
    Dense(7, activation='softmax')  # or 'sigmoid' for binary classification
])
model.compile(
    optimizer='sgd',  # or 'sgd', 'rmsprop', etc.
    loss='sparse_categorical_crossentropy',  # depends on your task
    metrics=['accuracy']  # optional, but helpful
)

C:\Users\pkitt\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [26]:
print(X.shape)  # Should be (num_samples, num_features)
print(y.shape)  # Should be (num_samples,)

(64, 6)
(64,)


In [27]:
import numpy as np

# Combine all batches into full training arrays
X_train = np.vstack([X for X, _ in batches])
y_train = np.hstack([y for _, y in batches])

model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
9375/9375 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - accuracy: 0.1885 - loss: 1.7874 - val_accuracy: 0.0000e+00 - val_loss: 6.0067
Epoch 2/5
9375/9375 ━━━━━━━━━━━━━━━━━━━━ 21s 1ms/step - accuracy: 0.1899 - loss: 1.7436 - val_accuracy: 0.0000e+00 - val_loss: 5.5566
Epoch 3/5
9375/9375 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.1899 - loss: 1.7435 - val_accuracy: 0.0000e+00 - val_loss: 6.6402
Epoch 4/5
9375/9375 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.1893 - loss: 1.7433 - val_accuracy: 0.0000e+00 - val_loss: 5.9704
Epoch 5/5
9375/9375 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - accuracy: 0.1906 - loss: 1.7428 - val_accuracy: 0.0000e+00 - val_loss: 6.7986


In [28]:
# Filter rows where Fertilizer Name is 0
df_eval = df_train[df_train['Fertilizer Name'] == 0]

# Extract features and labels
X_eval = df_eval[feature_cols].values.astype('float32')
y_eval = df_eval['Fertilizer Name'].values

In [29]:
for i, (X_batch, y_batch) in enumerate(batches):
    X_batch = X_batch.astype('float32')
    metrics = model.train_on_batch(X_batch, y_batch)
    print(f"Batch {i+1}: Loss = {metrics[0]:.4f}, Accuracy = {metrics[1]*100:.2f}%")

Batch 1: Loss = 6.7964, Accuracy = 0.00%
Batch 2: Loss = 6.7942, Accuracy = 0.04%
Batch 3: Loss = 6.7918, Accuracy = 0.09%
Batch 4: Loss = 6.7894, Accuracy = 0.13%
Batch 5: Loss = 6.7868, Accuracy = 0.17%
Batch 6: Loss = 6.7840, Accuracy = 0.21%
Batch 7: Loss = 6.7812, Accuracy = 0.26%
Batch 8: Loss = 6.7784, Accuracy = 0.30%
Batch 9: Loss = 6.7755, Accuracy = 0.34%
Batch 10: Loss = 6.7727, Accuracy = 0.38%
Batch 11: Loss = 6.7698, Accuracy = 0.42%
Batch 12: Loss = 6.7669, Accuracy = 0.47%
Batch 13: Loss = 6.7641, Accuracy = 0.51%
Batch 14: Loss = 6.7612, Accuracy = 0.55%
Batch 15: Loss = 6.7584, Accuracy = 0.59%
Batch 16: Loss = 6.7555, Accuracy = 0.64%
Batch 17: Loss = 6.7527, Accuracy = 0.68%
Batch 18: Loss = 6.7498, Accuracy = 0.72%
Batch 19: Loss = 6.7469, Accuracy = 0.76%
Batch 20: Loss = 6.7441, Accuracy = 0.80%
Batch 21: Loss = 6.7413, Accuracy = 0.85%
Batch 22: Loss = 6.7384, Accuracy = 0.89%
Batch 23: Loss = 6.7356, Accuracy = 0.93%
Batch 24: Loss = 6.7327, Accuracy = 0.97%
B

In [30]:
from sklearn.utils import shuffle

df_shuffled = df_train.sample(frac=1).reset_index(drop=True)
X_all = df_shuffled[feature_cols].values.astype('float32')
y_all = df_shuffled[label_col].values

loss, accuracy = model.evaluate(X_all, y_all)
print(f"New Accuracy: {accuracy * 100:.2f}%")

23438/23438 ━━━━━━━━━━━━━━━━━━━━ 32s 1ms/step - accuracy: 0.1230 - loss: 12.7098
New Accuracy: 12.31%


In [31]:
def ordered_classwise_batches(df, feature_cols, label_col, batch_size=32):
    batches = []
    class_labels = sorted(df[label_col].unique())

    for label in class_labels:
        group = df[df[label_col] == label].reset_index(drop=True)
        for i in range(0, len(group), batch_size):
            batch = group.iloc[i:i + batch_size]
            X_batch = batch[feature_cols].values
            y_batch = batch[label_col].values
            batches.append((X_batch, y_batch))

    return batches

In [32]:
from sklearn.utils import shuffle

df_mixed = shuffle(df_train).reset_index(drop=True)
X = df_mixed[feature_cols].values.astype('float32')
y = df_mixed[label_col].values

model.fit(X, y, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
9375/9375 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - accuracy: 0.1501 - loss: 1.9552 - val_accuracy: 0.1525 - val_loss: 1.9424
Epoch 2/5
9375/9375 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.1528 - loss: 1.9426 - val_accuracy: 0.1531 - val_loss: 1.9424
Epoch 3/5
9375/9375 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.1528 - loss: 1.9424 - val_accuracy: 0.1532 - val_loss: 1.9422
Epoch 4/5
9375/9375 ━━━━━━━━━━━━━━━━━━━━ 16s 2ms/step - accuracy: 0.1540 - loss: 1.9422 - val_accuracy: 0.1516 - val_loss: 1.9423
Epoch 5/5
9375/9375 ━━━━━━━━━━━━━━━━━━━━ 15s 2ms/step - accuracy: 0.1525 - loss: 1.9425 - val_accuracy: 0.1529 - val_loss: 1.9422


In [33]:
# Sanity check on label spread
print(np.unique(y, return_counts=True))

# Try predicting from untrained model
sample_preds = model.predict(X[:5])
print("Predictions:", sample_preds.argmax(axis=1))

(array([0, 1, 2, 3, 4, 5, 6]), array([113887, 114436, 112453, 110889, 111158,  94860,  92317],
      dtype=int64))
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Predictions: [0 0 4 4 4]


In [48]:
model.fit(X, y, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
9375/9375 ━━━━━━━━━━━━━━━━━━━━ 11s 1ms/step - accuracy: 0.1540 - loss: 1.9425 - val_accuracy: 0.1526 - val_loss: 1.9421
Epoch 2/5
9375/9375 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - accuracy: 0.1546 - loss: 1.9422 - val_accuracy: 0.1546 - val_loss: 1.9421
Epoch 3/5
9375/9375 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - accuracy: 0.1544 - loss: 1.9422 - val_accuracy: 0.1534 - val_loss: 1.9419
Epoch 4/5
9375/9375 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - accuracy: 0.1539 - loss: 1.9424 - val_accuracy: 0.1548 - val_loss: 1.9420
Epoch 5/5
9375/9375 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step - accuracy: 0.1534 - loss: 1.9422 - val_accuracy: 0.1552 - val_loss: 1.9420


In [35]:
y_pred = model.predict(X).argmax(axis=1)
np.bincount(y_pred)

23438/23438 ━━━━━━━━━━━━━━━━━━━━ 14s 587us/step


array([ 39350, 475036, 167426,   4409,  63779], dtype=int64)

In [36]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y)
class_weight_dict = dict(enumerate(class_weights))

print("Class Weights:", class_weight_dict)  # Check assigned weights

Class Weights: {0: 0.9407821537388564, 1: 0.9362688065194269, 2: 0.9527790022752363, 3: 0.9662171824333986, 4: 0.963878957365706, 5: 1.1294840516851903, 6: 1.160597258824021}


In [44]:
model.fit(X, y, epochs=5, batch_size=64, validation_split=0.2, class_weight=class_weight_dict)

Epoch 1/5
9375/9375 ━━━━━━━━━━━━━━━━━━━━ 26s 3ms/step - accuracy: 0.1481 - loss: 1.9456 - val_accuracy: 0.1458 - val_loss: 1.9455
Epoch 2/5
9375/9375 ━━━━━━━━━━━━━━━━━━━━ 26s 3ms/step - accuracy: 0.1435 - loss: 1.9457 - val_accuracy: 0.1502 - val_loss: 1.9449
Epoch 3/5
9375/9375 ━━━━━━━━━━━━━━━━━━━━ 27s 3ms/step - accuracy: 0.1453 - loss: 1.9456 - val_accuracy: 0.1520 - val_loss: 1.9452
Epoch 4/5
9375/9375 ━━━━━━━━━━━━━━━━━━━━ 25s 3ms/step - accuracy: 0.1460 - loss: 1.9456 - val_accuracy: 0.1437 - val_loss: 1.9456
Epoch 5/5
9375/9375 ━━━━━━━━━━━━━━━━━━━━ 24s 3ms/step - accuracy: 0.1451 - loss: 1.9456 - val_accuracy: 0.1408 - val_loss: 1.9462


In [38]:
y_pred = model.predict(X).argmax(axis=1)
print("New Predicted Class Distribution:", np.bincount(y_pred))

23438/23438 ━━━━━━━━━━━━━━━━━━━━ 14s 612us/step
New Predicted Class Distribution: [ 65693 348352  95153 171303    851  63371   5277]


In [39]:
# Sanity check on label spread
print(np.unique(y, return_counts=True))

# Try predicting from untrained model
sample_preds = model.predict(X[:5])
print("Predictions:", sample_preds.argmax(axis=1))

(array([0, 1, 2, 3, 4, 5, 6]), array([113887, 114436, 112453, 110889, 111158,  94860,  92317],
      dtype=int64))
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Predictions: [0 1 5 5 3]


In [40]:
for epoch in range(5):
    if epoch < 2:
        train_batches = classwise_batches  # Start with class-wise batches
    else:
        train_batches = mixed_batches  # Introduce mixed batches
    for X_batch, y_batch in train_batches:
        model.train_on_batch(X_batch.astype('float32'), y_batch)

NameError: name 'classwise_batches' is not defined

In [45]:
for i, (X_batch, y_batch) in enumerate(batches):
    X_batch = X_batch.astype('float32')
    metrics = model.train_on_batch(X,y)
    print(f"Batch {i+1}: Loss = {metrics[0]:.4f}, Accuracy = {metrics[1]*100:.2f}%")

Batch 1: Loss = 1.9460, Accuracy = 14.13%
Batch 2: Loss = 1.9460, Accuracy = 14.16%
Batch 3: Loss = 1.9459, Accuracy = 14.19%
Batch 4: Loss = 1.9459, Accuracy = 14.21%
Batch 5: Loss = 1.9458, Accuracy = 14.24%
Batch 6: Loss = 1.9458, Accuracy = 14.26%
Batch 7: Loss = 1.9457, Accuracy = 14.29%
Batch 8: Loss = 1.9457, Accuracy = 14.31%
Batch 9: Loss = 1.9457, Accuracy = 14.34%
Batch 10: Loss = 1.9456, Accuracy = 14.37%
Batch 11: Loss = 1.9456, Accuracy = 14.39%
Batch 12: Loss = 1.9456, Accuracy = 14.41%
Batch 13: Loss = 1.9455, Accuracy = 14.43%
Batch 14: Loss = 1.9455, Accuracy = 14.45%
Batch 15: Loss = 1.9454, Accuracy = 14.48%
Batch 16: Loss = 1.9454, Accuracy = 14.49%
Batch 17: Loss = 1.9454, Accuracy = 14.51%
Batch 18: Loss = 1.9454, Accuracy = 14.53%
Batch 19: Loss = 1.9453, Accuracy = 14.55%
Batch 20: Loss = 1.9453, Accuracy = 14.57%
Batch 21: Loss = 1.9453, Accuracy = 14.59%
Batch 22: Loss = 1.9452, Accuracy = 14.61%
Batch 23: Loss = 1.9452, Accuracy = 14.62%
Batch 24: Loss = 1.9

KeyboardInterrupt: 